In [61]:
import pandas as pd
import warnings
import tkinter as tk
warnings.filterwarnings("ignore")

In [82]:
cols="user_id item_id rating timestamp".split()
df = pd.read_csv('u.data',sep='\t',names=cols)
cols_movies = ['item_id', 'title']+[str(i) for i in range(22)]
movie_titles = pd.read_csv('u.item',sep='|',names=cols_movies, encoding='latin-1')
movie_titles = movie_titles[['item_id','title']]
data = pd.merge(df,movie_titles, on='item_id')
rating = pd.DataFrame(data.groupby('title')['rating'].mean())
rating['count'] = data['title'].value_counts()
pivot_df = data.pivot_table(index = 'user_id', columns = 'title', values = 'rating')



app=tk.Tk()
app.title('Recommendation System')
app.geometry('400x350')
app.configure(background="cyan")
 
tk.Label(app,text="RECOMMENDER SYSTEM",fg='white',bg='purple').place(x=65,y=1)
tk.Label(app,text="Movie You Watched",fg='white',bg='black').place(x=25,y=45)
tk.Label(app,text="Recommend For You",fg='white',bg='black').place(x=25,y=120)
tk.Label(app,text="Also Try",fg='white',bg='black').place(x=25,y=155)

movie_var=tk.Variable(app)
recommend_var=tk.Variable(app)
also_try_var=tk.Variable(app)

recommend_var.set('Type Movie Name')
also_try_var.set('Type Movie Name')


tk.Entry(app,textvariable=movie_var,width=25).place(x=150,y=45)
tk.Label(app,textvariable=recommend_var,width=25).place(x=150,y=120)
tk.Label(app,textvariable=also_try_var,width=25).place(x=150,y=155)    
         

def p():
         movie=movie_var.get().lower().strip()
         if movie:
         
            try:
                movie=movie_titles['title'][movie_titles['title'].apply(lambda x: movie in x.lower())].values[0]
                print(movie)
                movie_var.set(movie)
            except IndexError:
                recommend_var.set("Movie not found")
                recommend_var.set("Movie not found")
            else:
                corr_df = pd.DataFrame(pivot_df.corrwith(pivot_df[movie]), columns = ['Correlation'])
                corr_df.dropna(inplace=True)
                corr_df = corr_df.join(rating['count'])
                recommend_var.set(corr_df[(corr_df['count']>200)&(corr_df['Correlation']>0.4)].sort_values(by='Correlation',ascending=False).index[1])
                also_try_var.set(corr_df[corr_df['Correlation']>0.4].sort_values(by="Correlation",ascending=False).index[0])
         
       
       
         
tk.Button(app,text='Find Suggestion',command=p,bg='#AF0024',fg='white').place(x=110,y=80)
app.mainloop()        

Star Wars (1977)
